In [52]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats
import yfinance as yf #Mercado de valores
import requests
import seaborn as sns
import chardet
from sklearn.impute import SimpleImputer, KNNImputer

In [ ]:
Seccion de Javier..


In [ ]:
Seccion Brandon...

In [5]:
df = pd.read_csv('datasets/bank_marketing.csv')
df

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,nov,30,334,1,0,nonexistent,94.767,1.028,yes
41184,41184,46,blue-collar,married,professional.course,no,no,nov,6,383,1,0,nonexistent,94.767,1.028,no
41185,41185,56,retired,married,university.degree,no,yes,nov,24,189,2,0,nonexistent,94.767,1.028,no
41186,41186,44,technician,married,professional.course,no,no,nov,17,442,1,0,nonexistent,94.767,1.028,yes


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

Identificar valores perdidos


Renombrar columnas

In [ ]:
df.rename(columns={
    'credit_default': 'credit_base',
    'education': 'nivel_educativo',
    'age': 'edad'
}, inplace=True)

Separar columnas numericas

In [8]:
col_numericas = df.select_dtypes(include=[np.number]).columns.to_list()
col_numericas

['client_id',
 'age',
 'day',
 'contact_duration',
 'number_contacts',
 'previous_campaign_contacts',
 'cons_price_idx',
 'euribor_three_months']

In [10]:
col_categoricas = df.select_dtypes(exclude=[np.number]).columns.to_list()
col_categoricas

['job',
 'marital',
 'education',
 'credit_default',
 'mortgage',
 'month',
 'previous_outcome',
 'campaign_outcome']

In [13]:
for i in col_categoricas:
    df[i] = df[i].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   client_id                   41188 non-null  int64   
 1   age                         41188 non-null  int64   
 2   job                         41188 non-null  category
 3   marital                     41188 non-null  category
 4   education                   41188 non-null  category
 5   credit_default              41188 non-null  category
 6   mortgage                    41188 non-null  category
 7   month                       41188 non-null  category
 8   day                         41188 non-null  int64   
 9   contact_duration            41188 non-null  int64   
 10  number_contacts             41188 non-null  int64   
 11  previous_campaign_contacts  41188 non-null  int64   
 12  previous_outcome            41188 non-null  category
 13  cons_price_idx  

Resumen de las variables categoricas

In [19]:
resumen_categorica = {}

for i in col_categoricas:
    resumen_categorica[i] = df[i].value_counts()

for i,j in resumen_categorica.items():
    print(f'\nResumen de la columna {i}: ')
    print(j)


Resumen de la columna job: 
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

Resumen de la columna marital: 
marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

Resumen de la columna education: 
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

Resumen de la columna credit_default: 
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

Resumen de la columna mortgage: 
mortgage
yes        21576
no         18622
unknown      9

Creando los intervalos propios

In [25]:
intervalo = [0, 20, 30, 40, 50, 60, 70, 80, np.inf]
etiquetas = ['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']

grupo_edad = pd.cut(df['age'], bins=intervalo, labels=etiquetas)
grupo_edad

0        50-59
1        50-59
2        30-39
3        30-39
4        50-59
         ...  
41183    70-79
41184    40-49
41185    50-59
41186    40-49
41187    70-79
Name: age, Length: 41188, dtype: category
Categories (8, object): ['0-19' < '20-29' < '30-39' < '40-49' < '50-59' < '60-69' < '70-79' < '80+']

In [26]:
df.insert(2, 'grupo_edad', grupo_edad)
df

,client_id,age,grupo_edad,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,50-59,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,50-59,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,30-39,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,30-39,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,50-59,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,70-79,retired,married,professional.course,no,yes,nov,30,334,1,0,nonexistent,94.767,1.028,yes
41184,41184,46,40-49,blue-collar,married,professional.course,no,no,nov,6,383,1,0,nonexistent,94.767,1.028,no
41185,41185,56,50-59,retired,married,university.degree,no,yes,nov,24,189,2,0,nonexistent,94.767,1.028,no
41186,41186,44,40-49,technician,married,professional.course,no,no,nov,17,442,1,0,nonexistent,94.767,1.028,yes


Limpieza y manipulacion de datos

In [45]:
# Generar datos sintéticos
n_rows = 1000
data = {
    'id': range(1, n_rows + 1),
    'pais': np.random.choice(['Colombia', 'Argentina', 'Mexico', 'Chile', np.nan], n_rows),
    'productividad': np.append(np.random.normal(50, 10, n_rows - 5), [200, 220, 240, 260, 280]),  # Añadir outliers
    'edad': np.append(np.random.normal(35, 10, n_rows - 10), [np.nan] * 10),  # Añadir valores faltantes
    'fecha': np.random.choice(pd.date_range('2020-01-01', '2024-01-01'), n_rows)
}

df = pd.DataFrame(data)
df.head()

,id,pais,productividad,edad,fecha
0,1,Mexico,44.191251,40.146266,2023-01-04
1,2,Argentina,52.379006,41.172809,2020-02-24
2,3,Chile,60.037343,29.962955,2020-11-11
3,4,Argentina,58.865241,35.349067,2023-03-30
4,5,nan,45.808121,32.515846,2023-08-30


In [46]:
# Introducir valores faltantes aleatoriamente
for col in ['productividad', 'fecha']:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

In [47]:
df.to_csv('data/DataP.csv',index=False)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             1000 non-null   int64         
 1   pais           1000 non-null   object        
 2   productividad  900 non-null    float64       
 3   edad           990 non-null    float64       
 4   fecha          900 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 39.2+ KB


Eliminar filas

In [49]:
df1=df
df1.dropna(thresh=len(df1.columns)-1, inplace=True)

In [50]:
df1

,id,pais,productividad,edad,fecha
0,1,Mexico,44.191251,40.146266,2023-01-04
1,2,Argentina,52.379006,41.172809,2020-02-24
2,3,Chile,60.037343,29.962955,2020-11-11
3,4,Argentina,58.865241,35.349067,2023-03-30
4,5,nan,45.808121,32.515846,2023-08-30
...,...,...,...,...,...
995,996,nan,200.000000,NaN,2023-03-31
996,997,Colombia,220.000000,NaN,2021-05-01
997,998,Mexico,240.000000,NaN,2020-08-11
998,999,Colombia,260.000000,NaN,2022-01-18


eliminar columnas

In [ ]:
df1.dropna(axis=1, thresh=len(df1.columns)-80, inplace=True)

Imputacion de valores

In [54]:
mediaIP = SimpleImputer(strategy='mean')
df['productividad'] = mediaIP.fit_transform(df[['productividad']])
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 987 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             987 non-null    int64         
 1   pais           987 non-null    object        
 2   productividad  987 non-null    float64       
 3   edad           979 non-null    float64       
 4   fecha          900 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 46.3+ KB


KNNImputer

In [57]:
#knnIP = KNNImputer(n_neighbors=5)
mediaIpMF = SimpleImputer(strategy='most_frequent')
df1[['edad']] = mediaIpMF.fit_transform(df1[['edad']])
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 987 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             987 non-null    int64         
 1   pais           987 non-null    object        
 2   productividad  987 non-null    float64       
 3   edad           987 non-null    float64       
 4   fecha          900 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 46.3+ KB


In [60]:
df1['indicador_pais'] = df1['pais'].isnull().astype(int)
df1

,id,pais,productividad,edad,fecha,indicador_pais
0,1,Mexico,44.191251,40.146266,2023-01-04,0
1,2,Argentina,52.379006,41.172809,2020-02-24,0
2,3,Chile,60.037343,29.962955,2020-11-11,0
3,4,Argentina,58.865241,35.349067,2023-03-30,0
4,5,nan,45.808121,32.515846,2023-08-30,0
...,...,...,...,...,...,...
995,996,nan,200.000000,35.129394,2023-03-31,0
996,997,Colombia,220.000000,35.129394,2021-05-01,0
997,998,Mexico,240.000000,35.129394,2020-08-11,0
998,999,Colombia,260.000000,35.129394,2022-01-18,0


In [61]:
df['indicador_pais'].value_counts()

indicador_pais
0    987
Name: count, dtype: int64

# Class 04

In [62]:
file_path = 'datasets/obs_salud_corregida.csv'
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

encoding = result['encoding']
print(f"la codificacion del archivo es: {encoding}")


la codificacion del archivo es: ISO-8859-1


In [63]:
df = pd.read_csv(file_path, encoding=encoding)
df

,Año,Localidad de ocurrencia del caso,Sexo,Edad,Régimen aseguramiento en salud,Escolaridad,Ocupación,Tipo UTI,Clase UTI,Nivel de ingresos,Forma de Pago,Síntoma,Agente probablemente asociado,Tipo de lesión o Sistema Comprometido
0,2019,TUNJUELITO,Femenino,63,Contributivo,Secundaria incompleta,Zapateros y afines,Tipo II-Producción,Fija,1 SMMLV,A destajo,dificultad para respirar / ahogo,Exposición a otras sustancias químicas,Afecciones de vía respiratoria baja
1,2019,TUNJUELITO,Masculino,53,Contributivo,Secundaria completa,Zapateros y afines,Tipo II-Producción,Fija,1 SMMLV,A destajo,enrojecimiento eritema o irritación,Exposición a otras sustancias químicas,Alteraciones del tejido tegumentario
2,2019,USME,Masculino,44,Subsidiado,Primaria incompleta,Operadores de hornos de minerales y de hornos ...,Tipo II-Producción,Fija,Entre 1 y 2 SMMLV,A destajo,dificultad para respirar / ahogo,Exposición a otras sustancias tóxicas o venenosas,Afecciones de vía respiratoria baja
3,2019,TUNJUELITO,Masculino,72,Subsidiado,Primaria incompleta,Zapateros y afines,Tipo II-Producción,Fija,1 SMMLV,A destajo,dificultad para respirar / ahogo,Exposición a otras sustancias químicas,Afecciones de vía respiratoria baja
4,2019,CIUDAD BOLÍVAR,Masculino,56,Contributivo,Primaria completa,Montadores de estructuras metálicas,Tipo II-Producción,Fija,Entre 1 y 2 SMMLV,A destajo,congestión conjuntival: irritación enrojecimie...,Exposicion a gases humos y vapores,Trastornos del ojo y/o anexos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18748,2022,CIUDAD BOLÍVAR,Femenino,61,Subsidiado,Primaria incompleta,Oficios Varios,Tipo I-Servicios,Fija,Menos de 1 SMMLV,Monetaria fija,dolor en brazos,Movimientos repetitivos de MMSS,Otras mononeuropatías del MsSs
18749,2022,USME,Femenino,69,Contributivo,Primaria incompleta,Oficios Varios,Tipo I-Servicios,Fija,Menos de 1 SMMLV,Monetaria fija,dolor rodilla(s),Mvs.Repetitivos bipedestación prolongada,Mononeuropatías de MsIs y las no especificadas
18750,2022,USME,Femenino,60,Subsidiado,Secundaria incompleta,Niñeras y cuidadoras infantiles,Tipo I-Servicios,Fija,Menos de 1 SMMLV,Monetaria fija,dolor rodilla(s),Mvs.Repetitivos bipedestación prolongada,Mononeuropatías de MsIs y las no especificadas
18751,2023,CIUDAD BOLÍVAR,Femenino,52,Subsidiado,Primaria incompleta,Oficios Varios,Tipo I-Servicios,Fija,1 SMMLV,A destajo,hormigueo y adormecimiento en manos,Movimientos repetitivos de MMSS,Otras mononeuropatías del MsSs


In [64]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18753 entries, 0 to 18752
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Año                                    18753 non-null  int64 
 1   Localidad de ocurrencia del caso       18753 non-null  object
 2   Sexo                                   18753 non-null  object
 3   Edad                                   18753 non-null  int64 
 4   Régimen aseguramiento en salud         18753 non-null  object
 5   Escolaridad                            18753 non-null  object
 6   Ocupación                              18748 non-null  object
 7   Tipo UTI                               18753 non-null  object
 8   Clase UTI                              18753 non-null  object
 9   Nivel de ingresos                      18753 non-null  object
 10  Forma de Pago                          18753 non-null  object
 11  Síntoma        

In [65]:
df.describe()

,Año,Edad
count,18753.000000,18753.000000
mean,2020.519970,44.628433
std,1.724101,13.585965
min,2017.000000,15.000000
25%,2019.000000,34.000000
50%,2021.000000,45.000000
75%,2022.000000,55.000000
max,2023.000000,95.000000


In [70]:
categoricas = df.select_dtypes(exclude=[np.number]).columns.to_list()
print(categoricas)

for i in categoricas:
    df[i] = df[i].astype('category')

df.info()

['Localidad de ocurrencia del caso', 'Sexo', 'Régimen aseguramiento en salud', 'Escolaridad', 'Ocupación', 'Tipo UTI', 'Clase UTI', 'Nivel de ingresos', 'Forma de Pago', 'Síntoma', 'Agente probablemente asociado', 'Tipo de lesión o Sistema Comprometido']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18753 entries, 0 to 18752
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   Año                                    18753 non-null  int64   
 1   Localidad de ocurrencia del caso       18753 non-null  category
 2   Sexo                                   18753 non-null  category
 3   Edad                                   18753 non-null  int64   
 4   Régimen aseguramiento en salud         18753 non-null  category
 5   Escolaridad                            18753 non-null  category
 6   Ocupación                              18748 non-null  category
 7   Tipo UTI  

In [71]:
intervalo = [0, 20, 30, 40, 50, 60, 70, 80, np.inf]
etiquetas = ['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']

grupo_edad = pd.cut(df['Año'], bins=intervalo, labels=etiquetas)
df.insert(4, 'grupo_edad', grupo_edad)

In [72]:
df

,Año,Localidad de ocurrencia del caso,Sexo,Edad,grupo_edad,Régimen aseguramiento en salud,Escolaridad,Ocupación,Tipo UTI,Clase UTI,Nivel de ingresos,Forma de Pago,Síntoma,Agente probablemente asociado,Tipo de lesión o Sistema Comprometido
0,2019,TUNJUELITO,Femenino,63,80+,Contributivo,Secundaria incompleta,Zapateros y afines,Tipo II-Producción,Fija,1 SMMLV,A destajo,dificultad para respirar / ahogo,Exposición a otras sustancias químicas,Afecciones de vía respiratoria baja
1,2019,TUNJUELITO,Masculino,53,80+,Contributivo,Secundaria completa,Zapateros y afines,Tipo II-Producción,Fija,1 SMMLV,A destajo,enrojecimiento eritema o irritación,Exposición a otras sustancias químicas,Alteraciones del tejido tegumentario
2,2019,USME,Masculino,44,80+,Subsidiado,Primaria incompleta,Operadores de hornos de minerales y de hornos ...,Tipo II-Producción,Fija,Entre 1 y 2 SMMLV,A destajo,dificultad para respirar / ahogo,Exposición a otras sustancias tóxicas o venenosas,Afecciones de vía respiratoria baja
3,2019,TUNJUELITO,Masculino,72,80+,Subsidiado,Primaria incompleta,Zapateros y afines,Tipo II-Producción,Fija,1 SMMLV,A destajo,dificultad para respirar / ahogo,Exposición a otras sustancias químicas,Afecciones de vía respiratoria baja
4,2019,CIUDAD BOLÍVAR,Masculino,56,80+,Contributivo,Primaria completa,Montadores de estructuras metálicas,Tipo II-Producción,Fija,Entre 1 y 2 SMMLV,A destajo,congestión conjuntival: irritación enrojecimie...,Exposicion a gases humos y vapores,Trastornos del ojo y/o anexos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18748,2022,CIUDAD BOLÍVAR,Femenino,61,80+,Subsidiado,Primaria incompleta,Oficios Varios,Tipo I-Servicios,Fija,Menos de 1 SMMLV,Monetaria fija,dolor en brazos,Movimientos repetitivos de MMSS,Otras mononeuropatías del MsSs
18749,2022,USME,Femenino,69,80+,Contributivo,Primaria incompleta,Oficios Varios,Tipo I-Servicios,Fija,Menos de 1 SMMLV,Monetaria fija,dolor rodilla(s),Mvs.Repetitivos bipedestación prolongada,Mononeuropatías de MsIs y las no especificadas
18750,2022,USME,Femenino,60,80+,Subsidiado,Secundaria incompleta,Niñeras y cuidadoras infantiles,Tipo I-Servicios,Fija,Menos de 1 SMMLV,Monetaria fija,dolor rodilla(s),Mvs.Repetitivos bipedestación prolongada,Mononeuropatías de MsIs y las no especificadas
18751,2023,CIUDAD BOLÍVAR,Femenino,52,80+,Subsidiado,Primaria incompleta,Oficios Varios,Tipo I-Servicios,Fija,1 SMMLV,A destajo,hormigueo y adormecimiento en manos,Movimientos repetitivos de MMSS,Otras mononeuropatías del MsSs


Tablas de frecuencias, para las variables categoricas

In [74]:
df['Régimen aseguramiento en salud'] = df['Régimen aseguramiento en salud'].str.capitalize()
df['Escolaridad'] = df['Escolaridad'].str.capitalize()

In [75]:
ocupacion = df['Ocupación'].value_counts()
indice = ocupacion[ocupacion<100].index
indice

CategoricalIndex(['Lavadores de vehículos ventanas y afines',
                  'Vendedores en quioscos y puestos fijos de mercado',
                  'Obreros de la construcción de edificios',
                  'Lavanderos y planchadores manuales',
                  'Pintores empapeladores y afines',
                  'Sopladores modeladores laminadores cortadores y pulidores de vidrio',
                  'Operadores de máquinas para coser',
                  'Mecánicos y ajustadores de máquinas herramientas',
                  'Ayudantes en reparación y mecánica en general (excepto vehículos de motor)',
                  'Otros trabajadores de servicios personales a particulares no clasificados bajo otros epígrafes',
                  ...
                  'Porteros y guardianes y afines',
                  'Fabricantes y afinadores de instrumentos musicales',
                  'Preparadores y elaboradores de tabaco y sus productos',
                  'Profesionales de la informatica

In [76]:
df1 = df

In [77]:
df1['Ocupación'] = df1['Ocupación'].replace(indice, 'Otros oficios')
df1['Ocupación'].value_counts()

/var/folders/24/lxrdp9ld4dj69_0jp3rwwbx80000gn/T/ipykernel_4454/366332388.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df1['Ocupación'] = df1['Ocupación'].replace(indice, 'Otros oficios')


Ocupación
Vendedores y demostradores de tiendas almacenes y afines                2734
Mecánicos y ajustadores de vehículos de motor                           2638
Peluqueros especialistas en tratamientos de belleza y afines            2625
Otros oficios                                                           1653
Cocineros y afines                                                      1524
Oficios Varios                                                           784
Carpinteros (fabrican reparan muebles)                                   676
Sastres modistos costureros sombrereros y afines                         643
Vendedores ambulantes                                                    642
Encargados de control de abastecimiento e inventario                     537
Montadores de estructuras metálicas                                      497
Panaderos pasteleros y confiteros                                        423
Pintores barnizadores y enlacadores de artículos metálicos y afine

In [78]:

resumen = {}

for i in categoricas:
    resumen[i] = df1[i].value_counts()

for i,j in resumen.items():
    print(f'\nResumen de la columna {i}: ')
    print(j)


Resumen de la columna Localidad de ocurrencia del caso: 
Localidad de ocurrencia del caso
CIUDAD BOLÍVAR        1575
BARRIOS UNIDOS        1429
USME                  1293
SAN CRISTÓBAL         1238
SUBA                  1223
ENGATIVÁ              1218
KENNEDY               1174
RAFAEL URIBE URIBE    1170
FONTIBÓN              1146
BOSA                  1080
TUNJUELITO             956
LOS MÁRTIRES           954
USAQUÉN                943
ANTONIO NARIÑO         892
SANTA FE               888
PUENTE ARANDA          570
CHAPINERO              335
TEUSAQUILLO            294
LA CANDELARIA          288
SUMAPAZ                 87
Name: count, dtype: int64

Resumen de la columna Sexo: 
Sexo
Masculino        10282
Femenino          8441
Indeterminado       30
Name: count, dtype: int64

Resumen de la columna Régimen aseguramiento en salud: 
Régimen aseguramiento en salud
Contributivo    9241
Subsidiado      7025
No asegurado    2192
Vinculado        180
Especial         102
Excepción         13
